Recommender systems
================

Qué es?
=======

![](./images/mask.png)

- Sistema capaz de predecir el **rating** que un usuario dará a elementos de un conjunto
- Usualmente basado en tener ratings anteriores de él y de otros usuarios
- Una de las aplicaciones más importantes de machine learning
- Muy populares en los últimos años

### Aplicaciones reales

- Amazon / Mercado libre ...
- Netflix / Youtube / Spotify ...
- Facebook / Twitter / ...
- Google
- Banca/seguros
- ...

| Película         | Pedro | Emilia | Martín  | Paola  |
|------------------|:-----:| ------:|--------:|-------:|
| Plaga zombie     |     ? |      ? |      ?  |     ?  |
| Freddy vs Jason  |     ? |      ? |      ?  |     ?  |
| Saw 1            |     ? |      ? |      ?  |     ?  |
| Sintonía de amor |     ? |      ? |      ?  |     ?  |
| Titanic          |     ? |      ? |      ?  |     ?  |?


$n_u$ = cantidad de usuarios/personas

$n_m$ = cantidad de películas

$r(i,j)$ = 1 si el usuario **"j"** dejó su opinión sobre la película **"i"**, 0 si no

$y(i,j)$ = rating del usuario **"j"** sobre la película **"i"**

### Feedback: explícito o implícito?

- Muchas veces tenemos el feedback directamente: stars, +1, "me gusta", ...
- Pero ese datos muchas veces no está presente.

Generalmente se dispone de algún tipo de feedback, explícito o implícito: si el usuario vio un video completo o no, cantidad de veces que puso pausa o que volvió hacia atrás, número de veces que vio o accedió a un video, etc.

## Recomendaciones basadas en contenido

Se basa en la idea de que dos elementos parecidos entre sí deberían recibir opiniones similares por parte de un mismo usuario.

**No** conocemos el detalle de las preferencias de los usuarios (no se si a Pedro le gustan las de terror o las románticas).

**Sí** conocemos las características de cada película (si es de terror o romántica).

**Sí** conocemos algunas puntuaciones de los usuarios a algunas películas.

"Pedro no me dijo qué género le gusta, pero estas dos películas son de terror y dijo que le gustaron, así que le deben gustar las películas de terror".

| Película        | Pedro | Emilia | Martín  | Paola | X1 (terror) |  X2 (romántica) |
| ----------------|:-----:| ------:|--------:|-------|-------------|-----------------|
| Plaga zombie    |   5   |    5   |    0    |   0   |     0.9     |       0         |
| Freddy vs Jason |   5   |    ?   |    ?    |   0   |     1.0     |       0.01      |            
| Saw 1           |   ?   |    4   |    0    |   ?   |     0.99    |       0         |
| Sintonía de amor|   0   |    0   |    5    |   4   |     0.1     |       1.0       |
| Titanic         |   0   |    ?   |    5    |   ?   |     0       |       0.9       |

Por cada usuario voy a intentar aprender los parámetros de la recta que intentan replicar mejor los valores que tengo. Ejemplo:

$f_{pedro}(X) = X_1 * 5 + X_2 * 0 + 1$

5, 0 y 1 son los valores que tengo que encontrar, los **parámetros** para Pedro (θ = [5, 0, 1]).

La función recibe una película (como features X), y devuelve el rating que Pedro le da a esa película. La película es un vector de valores que la describen (en este ejemplo, "nivel de cada género").

Vamos a expresar esa función de recta en forma vectorial, como siempre:

$f_{pedro}(X) = X_1 * 5 + X_2 * 0 + 1$

se convierte en:

$f_{pedro}(X) = (\theta^{(pedro)})^T X$

**¿Cómo encontramos Θ?** Empezamos a probar al azar ?

Podemos tratar de armar una función de error: 

$f_{pedro}(x^{(i)}) - y^{(i,pedro)} $

O lo que es lo mismo:

$(\theta^{(pedro)})^T X - y^{(i,pedro)} $


"el raiting que predecimos de una película, cuánto difiere del rating que conocemos que Pedro le dio?")

Generalizada para el set entero de películas:

$E_{pedro} = \sum_{i:r(i,pedro)=1} ((\theta^{(pedro)})^T x^{(i)} - y^{(i,pedro)})^2$

"para todas las películas (i) donde tenemos el raiting de Pedro, cuánto difiere nuestro rating predicho del rating conocido?" (error cuadrático)

![](files/images/error_function.svg)

Una vez definida la función de error, cómo la usaríamos para que el modelo "aprenda" por sí solo?

## Descenso por el gradiente

![](files/images/gradient_descent.svg)

### Cuántas veces ? 1 o 4 ?

Tendría que armar una función por cada usuario? Tenemos que **entrenar un modelo por cada usuario**?? Por ahora parece que sí.

Podemos en vez redefinir la función de error e integrar las salidas para todos los usuarios, para tener **una sola función global, que sirve para todos los usuarios**:

$E_{total} = \sum_{j=1}^{n_u} \sum_{i:r(i,j)=1} ((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2$

"para todos los usuarios (j), para todas las películas (i), donde tengamos el raiting del usuario para esa película, cuánto difiere nuestro rating predicho del rating conocido?" (error cuadrático)

Ahora θ es un array, con los valores para cada usuario:

| Usuario |  θ(0)  |  θ(1)  |  θ(2) |
| --------| ------:|-------:|-------|
| Pedro   |    5   |    0   |   1   |
| Emilia  |    4   |    1   |   0   |
| Martín  |    0   |    4   |   0   |
| Paola   |    1   |    5   |   1   |


## Recomendaciones basadas en filtros colaborativos

Se basa en la idea de que dos personas que tuvieron opiniones parecidas en el pasado también las tendrán en el futuro.

**No** conocemos explicitamente el detalle de las preferencias de los usuarios.

**No** conocemos las características de cada película (si es de terror o romántica, o de qué género).

**Sí** conocemos algunas puntuaciones de los usuarios a algunas películas.

"Pedro no me dijo qué género le gusta y encima tampoco se de qué género es cada película, pero a él y a Emilia le suelen gustar las mismas películas. Entonces si esta película le gustó a Emilia, probablemente le guste también a Pedro".

| Película        | Pedro | Emilia | Martín  | Paola | X1 (terror) |  X2 (romántica) |
| ----------------|:-----:| ------:|--------:|-------|-------------|-----------------|
| Plaga zombie    |   5   |    5   |    0    |   0   |     ?       |       ?         |
| Freddy vs Jason |   5   |    ?   |    ?    |   0   |     ?       |       ?         |            
| Saw 1           |   ?   |    4   |    0    |   ?   |     ?       |       ?         |
| Sintonía de amor|   0   |    0   |    5    |   4   |     ?       |       ?         |
| Titanic         |   0   |    0   |    5    |   ?   |     ?       |       ?         |

#### Qué pasaría si las personas nos dijeran sus preferencias?


Si tenemos todos los vectores de parámetros ...

$\theta_1$ = [0,5,0]

$\theta_2$ = [0,5,0]

$\theta_3$ = [0,0,5]

$\theta_4$ = [0,0,5]

Pero no tenemos los vectores de las películas... cómo podría ser la función de error?

Podría ser igual!

$E_{total} = \sum_{j=1}^{n_u} \sum_{i:r(i,j)=1} ((\theta^{(j)})^T x^{(i)} - y^{(i,j)})^2$

"para todos los usuarios (j), para todas las películas (i), donde tengamos el raiting del usuario para esa película, cuánto difiere nuestro rating predicho del rating conocido?" (error cuadrático)

**Solo tenemos un cambio**: antes $\theta$ eran los pesos que no conocíamos y teníamos que aprender. Ahora esos ya los sabemos, están fijos, y lo que no conocemos y tenemos que aprender es...

$X!$

(los vectores que describen a cada película)

#### Qué pasaría si las personas NO nos dijeran sus preferencias?


* Podríamos utilizar los y(i, j) conocidos para aprender tanto las preferencias de los usuarios ($\theta$) como los vectores de las películas ($X$) al mismo tiempo. Misma función de error, y ahora tanto $\theta$ como $X$ son parámetros a aprender.

* Una vez que eso está entrenado, tenemos tanto preferencias de los usuarios como características del contenido, podemos hacer predicciones con nuestra función de siempre: $(\theta^{(pedro)})^T X - y^{(i,pedro)}$

## Problemas comúnes

- Cold start: nuevo usuario? nueva pelicula?
- Escalabilidad: En muchos casos podemos tener cientos de miles o millones de cosas para recomendar, lo cual hace que obtener recomendaciones de forma automática y al instante sea todo un desafío.
- Datos dispersos (sparse data): Generalmente tenemos data de una número mucho menor al total disponible de elementos.

## Actualidad: enfoque híbridos

Existe evidencia empírica de que combinar ambos modelos puede ser una buena alternativa para obtener resultados mejores que si se utilizaran de forma aislada.

Existen distintas principalmente dos formas de combinarlos:
- Combinando las salidas de ambos.
- Creando un único modelo que combine ambos enfoques.

Un caso de ejemplo es Netflix, que realiza recomendaciones de películas a partir del historial de búsqueda y visitas de otros usuarios (filtros colaborativos) y de películas que tienen características similares (basados en contenido).

## Métricas

Definir medidas de performance varía en relación a lo que se hace para un problema de clasificación común y corriente.

¿Que hagamos una recomendación y que el usuario no la haya seleccionado, significa que estaba mal?

Para medición offline se suelen usar metricas de ranking, entre ellas: 
* Precision at *n* (P@n) ("de los primeros N contenidos que recomendamos, cuáles realmente están en el top N de lo que le gusta al usuario?")
* MAP (mean average precision) ("en promedio, qué tanto se parecen nuestras N recomendaciones para cada usuario, a las top N cosas que sabemos que le gustan?")

Surgen otras métricas:

- Diversidad: mide cuánto se parecen las recomendaciones entre sí (la idea subyacente es que a mayor diversidad, más probabilidades de acertar).
- Persistencia: mide el grado en el que ante situaciones semejantes, el criterio de selección se mantiene consistente.
- Privacidad: generalmente existen restricciones legales sobre la privacidad de los datos de los usuarios, lo cual limitan o condicionan la utilidad de un determinado sistema.
- Serendipity: Mide el grado en que una recomendación es predecible o "esperable".
- Coverage: a cuantos usuarios les puedo hacer recomendación